In [31]:
#Aula 28 - FUNCAO WINDOW

In [59]:
#seção imports
import findspark
import os
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window

In [33]:
from pyspark.sql import SparkSession

In [34]:
# Este seção cria o APPNAME e configura o formato de saida da função show()
spark = ( 
    SparkSession.builder
    .master("local")
    .appName('ralves_sess_window')
    .config ("spark.sql.repl.eagerEval.enabled",True)
    .getOrCreate()
)

In [35]:
spark

In [36]:
#comando para parar uma seção SPARK
# spark.stop()

In [37]:
#describe python version
from platform import python_version
print(python_version())

3.7.0


In [38]:
# Define fontes como ALIAS do diretório do drive C
#fontes = r'C:\Users\Desktop\ROGERIO\3-Treinamentos\Udemy\Pyspark_Programacao_Dalton\DATASETS'
#compras = 'COMPRAS.parquet'
#livros = 'LIVROS.parquet'
#autores= 'AUTORES.parquet'
# 
# Carregar arquivo - daria para informar apenas o nome do arquivo a partir do ALIAS do diretório ? Ex:
#compras = spark.read.parquet(f"{fontes}\{nome_arquivo}").select('id','data','cd_livro','cd_cliente').alias('compras')


In [66]:
# Define fontes como ALIAS do diretório do drive C
fontes = r'C:\Users\INEX\Desktop\ROGERIO\3-Treinamentos\Udemy\Pyspark_Programacao_Dalton\DATASETS'
compras = 'COMPRAS.parquet'
livros = 'LIVROS.parquet'
autores= 'AUTORES.parquet'

In [67]:
ft_compras = spark.read.parquet(f"{fontes}\{compras}").select('id','data','cd_livro','cd_cliente').alias('compras')
ft_livros = spark.read.parquet(f"{fontes}\{livros}").select('id','data_lancamento','preco').alias('livros')
ft_autores = spark.read.parquet(f"{fontes}\{autores}").alias('autores')

In [75]:
t = ft_compras.join(ft_livros,ft_compras.cd_livro==ft_livros.id).join(ft_autores, ft_livros.id == ft_autores.id).drop('livros.id','autores.id')

In [82]:
#to define window variables
window1=Window.orderBy('ft_compras.id')
window2=Window.partitionBy('ft_compras.cd_cliente').orderBy('ft_compras.data')
window3=Window.partitionBy('ft_autores.autor').orderBy('ft_livros.data_lancamento')

In [140]:
#to create dataframe
( df
    .withColumn('num_linha',F.row_number().over(window1))
    .withColumn('ordem_compra',F.row_number().over(window2))
    .withColumn('total_acumulado_cliente',F.round(F.sum('ft_livros.preco').over(window2),2))
)

AnalysisException: "cannot resolve '`ft_compras.id`' given input columns: [Média, Vendas, Data, Soma Cumulativa, Produto];;\n'Project [Produto#11451, Data#11452, Vendas#11453L, Soma Cumulativa#11458L, Média#11464, row_number() windowspecdefinition('ft_compras.id ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS num_linha#11471]\n+- Project [Produto#11451, Data#11452, Vendas#11453L, Soma Cumulativa#11458L, Média#11464]\n   +- Project [Produto#11451, Data#11452, Vendas#11453L, Soma Cumulativa#11458L, Média#11464, Média#11464]\n      +- Window [avg(Vendas#11453L) windowspecdefinition(Produto#11451, Data#11452 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS Média#11464], [Produto#11451], [Data#11452 ASC NULLS FIRST]\n         +- Project [Produto#11451, Data#11452, Vendas#11453L, Soma Cumulativa#11458L]\n            +- Project [Produto#11451, Data#11452, Vendas#11453L, Soma Cumulativa#11458L]\n               +

In [139]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F
 
# Criando uma sessão Spark
spark = SparkSession.builder.appName("Window Functions Example").getOrCreate()
 
# Exemplo de dados
data = [
    ("Produto A", "2023-06-01", 1),
    ("Produto A", "2023-06-02", 2),
    ("Produto A", "2023-06-03", 3),
    ("Produto B", "2023-06-01", 4),
    ("Produto B", "2023-06-02", 5),
    ("Produto B", "2023-06-03", 6),
    ("Produto C", "2024-06-04", 7),
    ("Produto C", "2024-06-04", 8),
    ("Produto C", "2024-06-04", 9)
]
 
# Criando um DataFrame
df = spark.createDataFrame(data, ["Produto", "Data", "Vendas"])
 
# Definindo a janela
window_spec = Window.partitionBy("Produto").orderBy("Data").rowsBetween(Window.unboundedPreceding, Window.currentRow)
 
# Calculando a média móvel
df = (df
      .withColumn("Soma Cumulativa", F.sum("Vendas").over(window_spec))
      .withColumn("Média", F.avg("Vendas").over(window_spec))
     )

In [138]:
df.show()

+---------+----------+------+---------------+-----+
|  Produto|      Data|Vendas|Soma Cumulativa|Média|
+---------+----------+------+---------------+-----+
|Produto B|2023-06-01|     4|              4|  4.0|
|Produto B|2023-06-02|     5|              9|  4.5|
|Produto B|2023-06-03|     6|             15|  5.0|
|Produto C|2024-06-04|     7|              7|  7.0|
|Produto C|2024-06-04|     8|             15|  7.5|
|Produto C|2024-06-04|     9|             24|  8.0|
|Produto A|2023-06-01|     1|              1|  1.0|
|Produto A|2023-06-02|     2|              3|  1.5|
|Produto A|2023-06-03|     3|              6|  2.0|
+---------+----------+------+---------------+-----+

